Este notebook sirve para resumir los correos de una cuenta de Gmail que no han sido leídos.
Para ello se requiere crear una cuenta en la Consola de Google Cloud con el mismo correo que se va a ocupar para resumir los correos.
Se crea un nuevo proyecto en la Consola y se habilita la API de Gmail.
Posteriormente se crean unas credenciales OAuth 2.0 y se guradan las credenciales descargando el archivo credentials.json
Este archivo se guarda en la carpeta raiz del proyecto
Luego se instalan las librerías necesarias usando el siguiente comando:
pip install google-auth google-auth-oauthlib google-auth-httplib2 google-api-python-client

In [14]:
# imports

import base64
import os

from bs4 import BeautifulSoup
from dotenv import load_dotenv
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from IPython.display import Markdown, display
from openai import OpenAI

In [15]:
# Alcances necesarios para acceder a los correos
SCOPES = ["https://www.googleapis.com/auth/gmail.modify"]

# Cargamos las variables del fichero .env
load_dotenv()
api_key = os.getenv('OPENAI_API_KEY')
email_account = os.getenv("EMAIL")
# Check the key

if not api_key:
    print("No se encontró ninguna clave API: diríjase al cuaderno de resolución de problemas en esta carpeta para identificarla y solucionarla.")
elif not api_key.startswith("sk-proj-"):
    print("Se encontró una clave API, pero no inicia sk-proj-; verifique que esté usando la clave correcta; consulte el cuaderno de resolución de problemas")
elif api_key.strip() != api_key:
    print("Se encontró una clave API, pero parece que puede tener espacios o caracteres de tabulación al principio o al final; elimínelos; consulte el cuaderno de resolución de problemas")
else:
    print("¡Se encontró la clave API y hasta ahora parece buena!")


¡Se encontró la clave API y hasta ahora parece buena!


In [16]:
openai = OpenAI()

# Si esto no funciona, prueba con el menú Kernel >> Reiniciar Kernel y borrar las salidas de todas las celdas, luego ejecuta las celdas desde la parte superior de este cuaderno hacia abajo.
# Si TODAVÍA no funciona (¡qué horror!), consulta el cuaderno de resolución de problemas o prueba la siguiente línea:
# openai = OpenAI(api_key="your-key-here-starting-sk-proj-")

In [17]:
def authenticate_gmail():
    """Autentica al usuario con la API de Gmail y devuelve el servicio."""
    creds = None
    # Archivo token.json almacena las credenciales de acceso
    if os.path.exists("token.json"):
        creds = Credentials.from_authorized_user_file("token.json", SCOPES)
    # Si no hay credenciales válidas, inicia el flujo de autenticación
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file("credentials.json", SCOPES)
            creds = flow.run_local_server(port=0)
        # Guarda las credenciales para el futuro
        with open("token.json", "w") as token:
            token.write(creds.to_json())
    return build("gmail", "v1", credentials=creds)

In [18]:
def get_unread_emails(service):
    """Obtiene todos los correos no leídos."""
    results = (
        service.users()
        .messages()
        .list(userId="me", labelIds=["INBOX"], q="is:unread")
        .execute()
    )
    messages = results.get("messages", [])

    email_content = []

    if not messages:
        print("No hay correos no leídos.")
        return None

    for message in messages:
        msg = service.users().messages().get(userId="me", id=message["id"]).execute()
        payload = msg.get("payload", {})
        headers = payload.get("headers", [])
        subject = next(
            (header["value"] for header in headers if header["name"] == "Subject"),
            "Sin asunto",
        )
        body = ""

        # Intenta obtener el contenido del mensaje
        if "parts" in payload:
            for part in payload["parts"]:
                if part["mimeType"] == "text/plain":
                    data = part["body"].get("data")
                    if data:
                        body = base64.urlsafe_b64decode(data).decode()
                        break
        elif "body" in payload:
            data = payload["body"].get("data")
            if data:
                body = base64.urlsafe_b64decode(data).decode()

        email_content.append(f"Asunto: {subject}\n\n{body}")

        # Marca el mensaje como leído (opcional)
        service.users().messages().modify(
            userId="me", id=message["id"], body={"removeLabelIds": ["UNREAD"]}
        ).execute()

    return "\n\n".join(email_content)

In [19]:
# Define nuestro mensaje de sistema:

system_prompt = "Eres un asistente que analiza el contenido del correo electrónico \
y proporciona un breve resumen, ignorando el texto que podría estar relacionado con la navegación. \
Responder en Markdown."

In [20]:
def user_prompt_for(account_email):
    user_prompt = f"Estás viendo el correo electrónico no leido de la cuenta {account_email.title}"
    user_prompt += "\nEl contenido de estos correos es el siguiente; \
    proporciona un breve resumen de los correos en formato Markdown. \
    si el correo está en ingles haces el resumen en español \
    Si incluye noticias, productos o anuncios, resúmelos también.\n\n"
    user_prompt += account_email.text
    return user_prompt

In [21]:
def messages_for(account_email):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(account_email)},
    ]

In [22]:
def summarize(email: str):
    my_email = EmailAccount(email)
    response = openai.chat.completions.create(
        model="gpt-4o-mini", messages=messages_for(my_email)
    )
    return response.choices[0].message.content

In [23]:
def display_summary(email: str):
    summary = summarize(email)
    # print(summary)
    display(Markdown(summary))

In [24]:

class EmailAccount:
    """
    Una clase de utilidad para representar una cuenta de correo electrónico.
    """

    def __init__(self, email):
        """
        Crea este objeto de correo utilizando la biblioteca BeautifulSoup
        """
        service = authenticate_gmail()
        unread_emails = get_unread_emails(service)

        if unread_emails:
            soup = BeautifulSoup(unread_emails, "html.parser")
            self.title = f"Mensajes no leídos del correo: {email}"

            # Verifica si existe la etiqueta <body> antes de intentar limpiar elementos
            if soup.body:
                for irrelevant in soup.body.find_all(
                    ["script", "style", "img", "input"]
                ):
                    irrelevant.decompose()
                self.text = soup.body.get_text(separator="\n", strip=True)
            else:
                # Si no hay <body>, usa el texto directamente
                self.text = soup.get_text(separator="\n", strip=True)
        else:
            self.text = ""
            self.title = f"No hay mensajes no leídos en la cuenta: {email}"

In [25]:
display_summary(email_account)

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=427941571458-vi9ns96os8273vk2omijg1rft2rf6opv.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A59171%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fgmail.modify&state=vxjpJSsraSsSd0siWS2WQPmyzeHQXx&access_type=offline


## Resumen de Correos Electrónicos

### 1. **Asunto: 3 ways to make AI work for you (not against you)**
   - Se presentan tres cursos destacados y dos proyectos prácticos para mejorar habilidades en inteligencia artificial (IA), enfocados en:
     1. **Construcción de Agentes de IA**: Aprender a diseñar y gestionar sistemas de múltiples agentes.
     2. **Manual de IA Generativa**: Fundamentos sobre modelos de lenguaje (LLM) y cómo construir chatbots y aplicaciones multimodales.
     3. **Aprendizaje Automático con Herramientas Comunes**: Aprendizaje práctico usando frameworks como NumPy y TensorFlow.
   - **Proyectos destacados**:
     - **Clasificación de Sarcasmo** usando BERT.
     - **Generación de Música** con Inteligencia Artificial.
   - Además, se mencionan un nuevo curso sobre Azure Data Factory y un blog sobre preguntas comunes en entrevistas de IA.

### 2. **Asunto: Si no encuentras trabajo, te devolvemos tu dinero🤑**
   - Anuncio del **Programa de Aceleración Laboral en Ventas** que incluye formación intensiva, mentorías personalizadas y garantía de empleo. El programa está diseñado para capitalizar oportunidades laborales en ventas en LATAM.

### 3. **Asunto: What makes AI an essential tool for developers?**
   - Tabnine, una herramienta de IA para desarrollo, ha sido nominada como finalista en los **Tech Innovation CUBEd Awards**.
   - Destacan nuevas características, como la verificación de código para prevenir infracciones de propiedad intelectual.
   - Se anuncian sesiones en vivo para aprender sobre el uso adecuado de herramientas de IA.

### 4. **Asunto: Save 50% + 5% until Jan 31.**
   - Oferta de descuento del **50% + 5% adicional** en cursos de Educative, válida hasta finales de enero. Se anima a los usuarios a aprovechar las oportunidades de aprendizaje en una variedad de temas técnicos.

--- 

Estos resúmenes facilitan una visión clara y rápida de los contenidos y ofertas relevantes en los correos electrónicos no leídos.